#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

## Librerias

In [1]:
import pandas as pd
import numpy as np
import urllib.request
from PIL import Image
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

## Data

In [2]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 

train = pd.read_csv("data/train.csv")


## Data exploration

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_euros       912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB


In [4]:
train.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1104,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.4kg,387.0
1,114,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8GB,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.23kg,1379.0
2,172,Acer,Aspire A517-51G,Notebook,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,3kg,854.0
3,918,HP,Elitebook 820,Netbook,12.5,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.26kg,1483.0
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8GB,1TB HDD,AMD Radeon R4,Windows 10,2.8kg,519.0


## Data processing

Comienzo por la columana weight. La renombro con sus valor de unidad y limpio valores para que sean numericos

In [5]:
train = train.rename(columns={'Weight': 'Weight_Kg'})
train['Weight_Kg'] = train['Weight_Kg'].str[:-2]

Lo mismo para la columna ScreenResolution

In [6]:
# Extraer los últimos 9 digitos de la columna ScreenResolution y sobrescribir la columna original
train['ScreenResolution'] = train['ScreenResolution'].str[-9:]

# Creo las dos nuevas columnas 
train['SRx'] = 0
train['SRy'] = 0

# Creo dos nuevas columas y separo la columna original donde aparece x
train[['SRx', 'SRy']] = train['ScreenResolution'].str.split('x', expand=True)

# Convierto las dos nuevas columnas a tipo numérico
train['SRx'] = pd.to_numeric(train['SRx'])
train['SRy'] = pd.to_numeric(train['SRy'])

# Elimino la original 
del(train['ScreenResolution'])

#### Redondeo columna Inches

In [7]:
# Definir una funcion para redondear los valores y aplicar a columna Inches
def redondeo(x):
    if x % 1 >= 0.6:
        return int(x) + 1
    else:
        return int(x)
    
train['Inches'] = train['Inches'].apply(redondeo)

#### Trato Ram para hacerlo numerico

In [8]:
# Renombrar la columna Ram
train = train.rename(columns={'Ram': 'Ram_GB'})

# Quitar dos ultimos caracteres
train['Ram_GB'] = train['Ram_GB'].str[:-2]

# Convertir a tipo numerico
train['Ram_GB'] = train['Ram_GB'].astype(int)

#### Ahora trabajo con OpSys para agrupara por valores numericos

In [9]:
#Renombro para agrupar valores Operating system de mac

train['OpSys'].replace({'macOS': 'Mac OS', 'Mac OS X': 'Mac OS'}, inplace=True)

In [10]:

# Nuevos valores ---> 1 = Windows 10 | 2 = No OS | 3 = Linux | 4 = Windows 7| 5 = Chrome OS | 6 = Mac OS |  7 = Windows 10 S | 8 = Android



In [11]:
train['OpSys'].value_counts()

OpSys
Windows 10      751
No OS            47
Linux            40
Windows 7        31
Chrome OS        19
Mac OS           16
Windows 10 S      7
Android           1
Name: count, dtype: int64

#### Creo CPU_Brand
para agrupar por marcas de procesador

In [12]:
# Renombrar la columna Cpu
train = train.rename(columns={'Cpu': 'CPU'})

train['CPU_Brand'] = np.where(train['CPU'].str.contains('Intel', regex=True), 'Intel',
                          np.where(train['CPU'].str.contains('Mac', regex=True), 'Mac', 'AMD'))


# Nuevos valores, 1 para intel. 0 = AMD

train['CPU_Brand'] = train['CPU_Brand'].replace('AMD', 0)
train['CPU_Brand'] = train['CPU_Brand'].replace('Intel', 1)

#### Creo Memory2

In [13]:
# Creo feature 'Memory2' y asigno valores 0 = solo tiene una memoria. 1 tiene 2 memorias

train['Memory2'] = 0
train.loc[train['Memory'].str.contains('\+'), 'Memory2'] = 1

In [14]:
train['Memory2'].value_counts()

Memory2
0    773
1    139
Name: count, dtype: int64

## Pasos que si o si debéis realizar para poder participar en la competición:

In [15]:
train.head()

,laptop_ID,Company,Product,TypeName,Inches,CPU,Ram_GB,Memory,Gpu,OpSys,Weight_Kg,Price_euros,SRx,SRy,CPU_Brand,Memory2
0,1104,Acer,Aspire ES1-523,Notebook,15,AMD A8-Series 7410 2.2GHz,4,500GB HDD,AMD Radeon R5,Windows 10,2.4,387.0,1366,768,0,0
1,114,Dell,XPS 13,Ultrabook,13,Intel Core i7 7560U 2.4GHz,8,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.23,1379.0,3200,1800,1,0
2,172,Acer,Aspire A517-51G,Notebook,17,Intel Core i5 8250U 1.6GHz,8,256GB SSD,Nvidia GeForce MX150,Windows 10,3,854.0,1920,1080,1,0
3,918,HP,Elitebook 820,Netbook,12,Intel Core i7 7500U 2.7GHz,8,512GB SSD,Intel HD Graphics 620,Windows 10,1.26,1483.0,1920,1080,1,0
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17,AMD A6-Series 9220 2.5GHz,8,1TB HDD,AMD Radeon R4,Windows 10,2.8,519.0,1600,900,0,0


### 1. Definir X e y

In [16]:
X = train.drop(['Price_euros','Company','Product','TypeName','CPU','Memory','OpSys','Gpu','CPU' ], axis=1)
y = train['Price_euros'].copy()
X.shape

(912, 8)

In [17]:
y.shape

(912,)

### 2. Dividir X_train, X_test, y_train, y_test

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=422, shuffle=False)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [20]:
X_train

,laptop_ID,Inches,Ram_GB,Weight_Kg,SRx,SRy,CPU_Brand,Memory2
25,26,15,4,2.3,1366,768,1,0
84,776,15,16,2.72,3840,2160,1,1
10,1263,15,16,2.5,1920,1080,1,1
342,182,15,8,1.90,1920,1080,1,1
890,321,13,16,1.29,3840,2160,1,0
...,...,...,...,...,...,...,...,...
106,394,15,8,2,1920,1080,1,0
270,441,15,16,2.5,1920,1080,1,0
860,955,17,4,2.8,1600,900,1,0
435,210,15,6,2.3,1366,768,1,0


### 3. Asignar el modelo (vacío) a una variable


In [21]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [22]:
model.fit(X_train, y_train)

LinearRegression()

In [23]:
predictions = model.predict(X_test)
predictions

array([1087.37255269,  152.41697359,  973.97048142, 1144.77907883,
       1919.81779853,  571.37156222,  829.74756468, 1547.69138658,
        981.59506259, 1160.83356565, 1237.30721184,  522.56571418,
        611.82934261, 1130.70689924, 1453.99597049, 1759.32829396,
       1823.23710048, 1496.68733536,  997.44206238,  603.3620068 ,
        664.36101434,  521.95687565,  810.58705112, 1146.59869153,
       1072.64530416,  691.9013134 ,  861.29120617, 1653.89480506,
        977.94852377,  796.73942308, 1108.61479375, 1064.92291554,
        978.28604018, 1288.22254661, 1173.89558583,  461.92254785,
        969.01787356, 1336.20500934, 1174.53687519, 1173.25368438,
       1173.937979  , 1017.72976665, 1136.58996789, 1908.579441  ,
       1156.73894886,  530.08893169, 1785.52137026,  571.15296062,
        997.9626907 ,  838.56037608, 1911.52213966, 1091.23811078,
        202.56538511, 1621.80671266,  880.4542845 , 1152.36178864,
       1906.88355152, 1036.41546123, 2016.30776937, 1874.33133

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
print('RMSE: ', np.sqrt(mean_squared_error(y_test, predictions)))

RMSE:  465.98382870905084


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [26]:
pred = pd.read_csv("data/test.csv")
pred.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,750,Toshiba,Tecra X40-D-10G,Notebook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.25kg
1,726,Mediacom,Smartbook 142,Notebook,14.0,IPS Panel Full HD 1920x1200,Intel Atom Z8350 1.92GHz,4GB,32GB HDD,Intel HD Graphics 400,Windows 10,1.4kg
2,633,Mediacom,SmartBook 140,Notebook,14.0,Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,2GB,32GB Flash Storage,Intel HD Graphics,Windows 10,1.4kg
3,363,Dell,Inspiron 7577,Gaming,15.6,Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8GB,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.65kg
4,319,Asus,VivoBook Flip,2 in 1 Convertible,11.6,Touchscreen 1366x768,Intel Celeron Dual Core N3350 1.1GHz,2GB,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.1kg


### Transformaciones X_pred

In [27]:
train = pred.rename(columns={'Weight': 'Weight_Kg'})
pred['Weight_Kg'] = train['Weight_Kg'].str[:-2]

In [28]:
# Extraer los últimos 9 digitos de la columna ScreenResolution y sobrescribir la columna original
pred['ScreenResolution'] = pred['ScreenResolution'].str[-9:]

# Creo las dos nuevas columnas 
pred['SRx'] = 0
pred['SRy'] = 0

# Creo dos nuevas columas y separo la columna original donde aparece x
pred[['SRx', 'SRy']] = pred['ScreenResolution'].str.split('x', expand=True)

# Convierto las dos nuevas columnas a tipo numérico
pred['SRx'] = pd.to_numeric(pred['SRx'])
pred['SRy'] = pd.to_numeric(pred['SRy'])

# Elimino la original 
del(pred['ScreenResolution'])

In [29]:
pred['Inches'] = pred['Inches'].apply(redondeo)

In [30]:
# Renombrar la columna Ram
pred = pred.rename(columns={'Ram': 'Ram_GB'})

# Quitar dos ultimos caracteres
pred['Ram_GB'] = pred['Ram_GB'].str[:-2]

# Convertir a tipo numerico
pred['Ram_GB'] = pred['Ram_GB'].astype(int)



In [31]:
pred['OpSys'].replace({'macOS': 'Mac OS', 'Mac OS X': 'Mac OS'}, inplace=True)

In [32]:
# Renombrar la columna Cpu
pred = pred.rename(columns={'Cpu': 'CPU'})

pred['CPU_Brand'] = np.where(pred['CPU'].str.contains('Intel', regex=True), 'Intel',
                          np.where(pred['CPU'].str.contains('Mac', regex=True), 'Mac', 'AMD'))


# Nuevos valores, 1 para intel. 0 = AMD

pred['CPU_Brand'] = pred['CPU_Brand'].replace('AMD', 0)
pred['CPU_Brand'] = pred['CPU_Brand'].replace('Intel', 1)

In [33]:
# Creo feature 'Memory2' y asigno valores 0 = solo tiene una memoria. 1 tiene 2 memorias

pred['Memory2'] = 0
pred.loc[pred['Memory'].str.contains('\+'), 'Memory2'] = 1

In [34]:
pred.head()

,laptop_ID,Company,Product,TypeName,Inches,CPU,Ram_GB,Memory,Gpu,OpSys,Weight,Weight_Kg,SRx,SRy,CPU_Brand,Memory2
0,750,Toshiba,Tecra X40-D-10G,Notebook,14,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.25kg,1.25,1920,1080,1,0
1,726,Mediacom,Smartbook 142,Notebook,14,Intel Atom Z8350 1.92GHz,4,32GB HDD,Intel HD Graphics 400,Windows 10,1.4kg,1.4,1920,1200,1,0
2,633,Mediacom,SmartBook 140,Notebook,14,Intel Atom x5-Z8350 1.44GHz,2,32GB Flash Storage,Intel HD Graphics,Windows 10,1.4kg,1.4,1920,1080,1,0
3,363,Dell,Inspiron 7577,Gaming,15,Intel Core i5 7300HQ 2.5GHz,8,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.65kg,2.65,1920,1080,1,0
4,319,Asus,VivoBook Flip,2 in 1 Convertible,11,Intel Celeron Dual Core N3350 1.1GHz,2,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.1kg,1.1,1366,768,1,0


In [35]:
X2 = pred.drop(['Company','Product','Weight','TypeName','CPU','Memory','OpSys','Gpu','CPU' ], axis=1)

In [36]:
X2.shape

(391, 8)

**¿Por qué da error?** 

In [37]:
predictions_submit = model.predict(X2)
predictions_submit

array([ 1.15342985e+03,  9.45493701e+02,  6.15266812e+02,  1.00109108e+03,
        3.79248860e+02,  6.32115733e+02,  6.62840460e+02,  1.79427302e+03,
        1.69544352e+03,  1.21157212e+03,  9.91289185e+02,  1.04192542e+03,
        1.14795130e+03,  8.02271599e+02,  6.57549306e+02,  1.11802577e+03,
        1.05085184e+03,  1.77256439e+03,  7.14991615e+02,  1.12861688e+03,
        1.18336823e+03,  1.07496583e+03,  6.33084499e+02,  9.88152880e+02,
        5.91441263e+02,  1.38808639e+03,  8.16457188e+02,  9.78892228e+02,
        4.64879739e+02,  1.17311771e+03,  1.08904899e+03,  1.16224775e+03,
        9.49871306e+02,  1.23595156e+03,  1.14173913e+03,  1.12211424e+03,
        1.20119942e+03,  1.15351406e+03,  8.79001345e+02,  1.81807933e+03,
        1.74240625e+03,  1.04930114e+03,  5.27783771e+02,  1.44883387e+03,
        9.82272002e+02,  1.80054291e+03,  6.68507283e+02,  1.78071027e+03,
        1.11542907e+03,  1.82179390e+03,  8.04183889e+02,  6.74043963e+02,
        1.01139166e+03,  

IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [38]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

In [39]:
#X_pred = pd.read_csv("data/test.csv", index_col=0)
#X_pred.head()

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [40]:
predictions_submit = model.predict(X2)
predictions_submit

array([ 1.15342985e+03,  9.45493701e+02,  6.15266812e+02,  1.00109108e+03,
        3.79248860e+02,  6.32115733e+02,  6.62840460e+02,  1.79427302e+03,
        1.69544352e+03,  1.21157212e+03,  9.91289185e+02,  1.04192542e+03,
        1.14795130e+03,  8.02271599e+02,  6.57549306e+02,  1.11802577e+03,
        1.05085184e+03,  1.77256439e+03,  7.14991615e+02,  1.12861688e+03,
        1.18336823e+03,  1.07496583e+03,  6.33084499e+02,  9.88152880e+02,
        5.91441263e+02,  1.38808639e+03,  8.16457188e+02,  9.78892228e+02,
        4.64879739e+02,  1.17311771e+03,  1.08904899e+03,  1.16224775e+03,
        9.49871306e+02,  1.23595156e+03,  1.14173913e+03,  1.12211424e+03,
        1.20119942e+03,  1.15351406e+03,  8.79001345e+02,  1.81807933e+03,
        1.74240625e+03,  1.04930114e+03,  5.27783771e+02,  1.44883387e+03,
        9.82272002e+02,  1.80054291e+03,  6.68507283e+02,  1.78071027e+03,
        1.11542907e+03,  1.82179390e+03,  8.04183889e+02,  6.74043963e+02,
        1.01139166e+03,  

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [41]:
sample = pd.read_csv("data/sample_submission.csv")

In [42]:
sample.head()

,laptop_ID,Price_euros
0,750,500
1,726,500
2,633,500
3,363,650
4,319,650


In [43]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [44]:
predictions_submit  

array([ 1.15342985e+03,  9.45493701e+02,  6.15266812e+02,  1.00109108e+03,
        3.79248860e+02,  6.32115733e+02,  6.62840460e+02,  1.79427302e+03,
        1.69544352e+03,  1.21157212e+03,  9.91289185e+02,  1.04192542e+03,
        1.14795130e+03,  8.02271599e+02,  6.57549306e+02,  1.11802577e+03,
        1.05085184e+03,  1.77256439e+03,  7.14991615e+02,  1.12861688e+03,
        1.18336823e+03,  1.07496583e+03,  6.33084499e+02,  9.88152880e+02,
        5.91441263e+02,  1.38808639e+03,  8.16457188e+02,  9.78892228e+02,
        4.64879739e+02,  1.17311771e+03,  1.08904899e+03,  1.16224775e+03,
        9.49871306e+02,  1.23595156e+03,  1.14173913e+03,  1.12211424e+03,
        1.20119942e+03,  1.15351406e+03,  8.79001345e+02,  1.81807933e+03,
        1.74240625e+03,  1.04930114e+03,  5.27783771e+02,  1.44883387e+03,
        9.82272002e+02,  1.80054291e+03,  6.68507283e+02,  1.78071027e+03,
        1.11542907e+03,  1.82179390e+03,  8.04183889e+02,  6.74043963e+02,
        1.01139166e+03,  

In [45]:
sample

,laptop_ID,Price_euros
0,750,500
1,726,500
2,633,500
3,363,650
4,319,650
...,...,...
386,693,650
387,1132,500
388,49,650
389,1243,650


In [46]:
submission = pd.DataFrame({"laptop_ID": sample['laptop_ID'], "Price_euros": predictions_submit})

In [47]:
submission.head()

,laptop_ID,Price_euros
0,750,1153.429853
1,726,945.493701
2,633,615.266812
3,363,1001.091076
4,319,379.248860


In [48]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [49]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [50]:
chequeator(submission)

You're ready to submit!


In [51]:
submission

,laptop_ID,Price_euros
0,750,1153.429853
1,726,945.493701
2,633,615.266812
3,363,1001.091076
4,319,379.248860
...,...,...
386,693,1777.095280
387,1132,1916.141965
388,49,631.604545
389,1243,605.055445
